In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
     |████████████████████████████████| 6.8 MB 31.4 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 5.6 MB/s 
     |████████████████████████████████| 243 kB 38.8 MB/s 
     |████████████████████████████████| 133 kB 35.9 MB/s 
     |████████████████████████████████| 1.1 MB 27.5 MB/s 
     |████████████████████████████████| 144 kB 36.5 MB/s 
     |████████████████████████████████| 94 kB 1.5 MB/s 
     |████████████████████████████████| 271 kB 29.6 MB/s 


## Predict using pre-trained model

In [31]:
from transformers import pipeline
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
named_entity_recognizer = pipeline(
"ner", model=model, tokenizer=tokenizer, grouped_entities=True
)

loading configuration file https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dbfd3b2eb181c7b63cbbd8e7773a5e64941849440953d50ecad5ef346ad8286a.8f943745c8dd5e96d7b60c9b9e1be5711aff8aff42413b74288e076022e6e2bf
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MI

In [33]:
input_text = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge which is visible from the window."
result = named_entity_recognizer(input_text.lower())
result

[{'end': 12,
  'entity_group': 'ORG',
  'score': 0.83726805,
  'start': 8,
  'word': 'face'},
 {'end': 120,
  'entity_group': 'LOC',
  'score': 0.5114308,
  'start': 117,
  'word': '##hat'}]

In [36]:
for d in result:
  del d["start"], d["end"]

In [37]:
result

[{'entity_group': 'ORG', 'score': 0.83726805, 'word': 'face'},
 {'entity_group': 'LOC', 'score': 0.5114308, 'word': '##hat'}]

## Data

In [2]:
!wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/WNUT17/CONLL-format/data/train/wnut17train.conll -O train.conll
!wget https://raw.githubusercontent.com/leondz/emerging_entities_17/master/emerging.dev.conll -O val.conll

--2022-02-09 18:14:31--  https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/WNUT17/CONLL-format/data/train/wnut17train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491387 (480K) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>] 479.87K  --.-KB/s    in 0.03s   

2022-02-09 18:14:32 (13.5 MB/s) - ‘train.conll’ saved [491387/491387]

--2022-02-09 18:14:32--  https://raw.githubusercontent.com/leondz/emerging_entities_17/master/emerging.dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, aw

In [3]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

label_list = ['B-product',
 'I-location',
 'B-person',
 'B-corporation',
 'B-group',
 'I-person',
 'I-product',
 'I-corporation',
 'O',
 'I-group',
 'B-creative-work',
 'I-creative-work',
 'B-location']

task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [4]:
# Encode labels
label_encoding_dict = {}
for idx, label in enumerate(label_list):
  label_encoding_dict[label] = idx

In [5]:
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list] 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})

train_df = get_tokens_and_ner_tags("train.conll")
val_df = get_tokens_and_ner_tags("val.conll")

In [6]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

## Tokenize

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [8]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized_datasets = val_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Train

In [9]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer
)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [10]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3394
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 213


Epoch,Training Loss,Validation Loss
1,No log,0.319852


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens.
***** Running Evaluation *****
  Num examples = 1009
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=213, training_loss=0.19269225071293647, metrics={'train_runtime': 44.459, 'train_samples_per_second': 76.34, 'train_steps_per_second': 4.791, 'total_flos': 46123081338900.0, 'train_loss': 0.19269225071293647, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

In [11]:
trainer.save_model("model/")

Saving model checkpoint to model/
Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin
tokenizer config file saved in model/tokenizer_config.json
Special tokens file saved in model/special_tokens_map.json


## Predict using trained model

In [12]:
tokenizer = AutoTokenizer.from_pretrained('model/')

Didn't find file model/added_tokens.json. We won't load it.
loading file model/vocab.txt
loading file model/tokenizer.json
loading file None
loading file model/special_tokens_map.json
loading file model/tokenizer_config.json


In [13]:
model = AutoModelForTokenClassification.from_pretrained('model/')

loading configuration file model/config.json
Model config DistilBertConfig {
  "_name_or_path": "model/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_c

In [14]:
test_sentence = "Facebook is in New York"
test_tokens = tokenizer(test_sentence, return_tensors="pt")

In [20]:
predictions = model(**test_tokens)

In [21]:
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

words = tokenizer.batch_decode(test_tokens['input_ids'])

In [22]:
predictions

['O', 'B-corporation', 'O', 'O', 'B-location', 'B-location', 'O']

In [23]:
words

['[CLS] facebook is in new york [SEP]']